In [1]:
#filename = "ml.py"
#exec(compile(open(filename, "rb").read(), filename, 'exec'))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import dateutil.parser as parser
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from sklearn.cluster import KMeans

import tensorflow as tf

nltk.download('vader_lexicon')

# read the dataset into pandas dataframe
# df = pd.read_csv('./../downsampled_data', delim_whitespace=False).dropna()
df = pd.read_csv('./../downsampled_data.csv').dropna()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/trevorcarpenter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
df['categoryId'] = df['categoryId'].astype(np.float)

categories = pd.DataFrame(np.zeros((len(df), len(df['categoryId'].unique()))), columns = df['categoryId'].unique())
index = 0
for id in df['categoryId']:
    categories[id][index] = 1
    index += 1

In [5]:
# display the first five rows
retrieval_time = df['time_retrieved']
publish_time = df['publishedAt']
channel_publish_time = df['Channel_publishedAt']
retrieval_time_11_19_14 = df['11_19_14_update_timestamp']
columns_to_drop = ['Unnamed: 0', 'definition', 'categoryId', 'publishedAt', 'time_retrieved', 'Channel_title', '11_19_14_update_timestamp', 'Channel_publishedAt', 'video_id', 'channelId', 'thumbnail_link', 'Channel_country']
df = df.drop(columns_to_drop, axis = 1)

In [6]:
categories.columns

Float64Index([24.0, 25.0, 26.0, 17.0, 22.0, 20.0,  1.0, 10.0, 23.0, 28.0,  2.0,
              27.0, 19.0, 15.0, 29.0, 30.0],
             dtype='float64')

In [7]:
index = 0
for column in categories.columns:
    df.insert(df.shape[1], "Category_"+str(column), categories[column])
    index += 1

In [8]:
## time updates
import dateutil.parser as parser
age = []
age_update = []
channel_age = []
for i in df.index:
    channel_publish_time[i] = channel_publish_time[i].replace("\"", "")
    age.append(parser.isoparse(retrieval_time[i]) - parser.isoparse(publish_time[i]))
    age_update.append(parser.isoparse(retrieval_time_11_19_14[i]) - parser.isoparse(publish_time[i]))
    channel_age.append(parser.isoparse(channel_publish_time[i]) - parser.isoparse(publish_time[i]))
    if df['ratings_disabled'][i] == 'True':
        df['ratings_disabled'][i] = True
    elif df['ratings_disabled'][i] == 'False':
        df['ratings_disabled'][i] = False
        
    if df['Channel_hiddenSubscriberCount'][i] == 'True':
        df['Channel_hiddenSubscriberCount'][i] = True
    elif df['Channel_hiddenSubscriberCount'][i] == 'False':
        df['Channel_hiddenSubscriberCount'][i] = False
        
        
    if df['trended_later'][i] == 'True':
        df['trended_later'][i] = True
    elif df['trended_later'][i] == 'False':
        df['trended_later'][i] = False

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEn

In [9]:
## sentiment values

titles = df['title']
channel_title = df['channelTitle']
description = df['description']
channel_description = df['Channel_description']

title_sentiment_vals = []
channel_title_sentiment_vals = []
description_sentiment_vals = []
channel_description_sentiment_vals = []

sid = SentimentIntensityAnalyzer()
for sentence in titles:
    ss = sid.polarity_scores(str(sentence))
    title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_title:
    ss = sid.polarity_scores(str(sentence))
    channel_title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in description:
    ss = sid.polarity_scores(str(sentence))
    description_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_description:
    ss = sid.polarity_scores(str(sentence))
    channel_description_sentiment_vals.append(ss['pos']-ss['neg'])

In [10]:
df

,title,channelTitle,duration,trended_later,tags,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,...,Category_1.0,Category_10.0,Category_23.0,Category_28.0,Category_2.0,Category_27.0,Category_19.0,Category_15.0,Category_29.0,Category_30.0
0,Biden Victory Cold Open - SNL,Saturday Night Live,-0.0707230866960673,False,36.0,False,Joe Biden (Jim Carrey) and Donald Trump (Alec ...,0.819940,0.504608,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,How President Trump and the White House reacte...,CNBC Television,-0.0856798827419832,False,28.0,False,CNBC's Kayla Tausche joins The News with Shepa...,-0.274970,-0.350255,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Watch Live: President-Elect Joe Biden Addresse...,ABC News,0.09488158288091483,False,13.0,False,#Election2020 #Election #JoeBiden #PresidentEl...,0.650191,0.461214,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,I GOT PLASTIC SURGERY TO SEE HOW MY FRIENDS RE...,Alexa Rivera,-0.07104706061763949,False,0.0,False,"This was definitely such a fun prank, their re...",-0.258217,0.002973,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FC Barcelona 5 - 2 Real Betis - HIGHLIGHTS & G...,beIN SPORTS USA,-0.07288291283988188,False,12.0,False,Lionel Messi entró al segundo tiempo y resucit...,-0.330711,-0.407708,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,US Election: Recount likely as candidates figh...,Sky News,-0.08497793924524347,False,19.0,False,The fight for the White House has intensified ...,-0.192403,-0.192300,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,The Packers should trade for Dallas&#39; WR Am...,The Herd with Colin Cowherd,-0.07536671290526863,False,33.0,False,Trade rumors are swirling around Dallas Cowboy...,-0.269840,-0.361884,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877,Make You Mine This Season (Happiest Season),Tegan and Sara - Topic,-0.08692178277467658,False,1.0,False,Provided to YouTube by Facet/Warner Records Ma...,-0.340590,-0.440462,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
878,映画『ドクター・デスの遺産―BLACK FILE―』本編特別映像 2020年11月13日（金）公開,ワーナー ブラザース 公式チャンネル,-0.08395202182693154,False,35.0,False,綾野剛 × 北川景子 共演！ 警視庁捜査一課Ｎo.１コンビが挑む、 安楽死を手口にする連続殺...,-0.260114,-0.411267,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from sklearn import preprocessing

df['title'] = title_sentiment_vals
df['channelTitle'] = channel_title_sentiment_vals
df['description'] = description_sentiment_vals
df['Channel_description'] = channel_description_sentiment_vals

le = preprocessing.LabelEncoder()
df['trending?'] = le.fit_transform(df['trending?'])
df['ratings_disabled'] = le.fit_transform(df['ratings_disabled'])
df['Channel_hiddenSubscriberCount'] = le.fit_transform(df['Channel_hiddenSubscriberCount'])
pd.set_option('display.max_columns', None)
df = df.dropna()

In [12]:
original_df = df.copy()

In [56]:
df = original_df.copy()

In [57]:
# dummy values for now - get these interactively later...
num_hidden_layers = 4
num_hidden_layer_nodes = [20, 20, 30, 20]
train_ratio = .7
hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
optimizer = 'adam'
learning_rate = .005
loss = 'mean_squared_error'
metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
metrics_names = ["accuracy","recall","precision"]
epochs = 300
batch_size = 200
num_clusters = 3
original_trend_values = False

In [58]:
# Split the data into training and testing set by 70:30

In [59]:
# separate data into x and y - just random y for now..
y_column = 'trending?'
if not original_trend_values:
    df = df.drop(['trending?','trended_later'], axis = 1)
    clusters = KMeans(n_clusters=num_clusters).fit(df).labels_
    if 'cluster' in df.columns:
        df["cluster"]= clusters
    else:
        df.insert(df.shape[1], "cluster", clusters)
    y_column = 'cluster'
    
    
ratio = 0.7
train, test = train_test_split(df, train_size=ratio, random_state=42)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

train_X = train.loc[:,train.columns != y_column]
train_Y = train[y_column]
test_X = test.loc[:,test.columns != y_column]
test_Y = test[y_column]



new_train_Y = train_Y
new_test_Y = test_Y

if not original_trend_values:
    new_train_Y = []
    new_test_Y = []
    potential_y_values = np.zeros((num_clusters, num_clusters))
    for c in range(num_clusters):
        potential_y_values[c][c] = 1
    for index in range(len(train_Y)):
        new_train_Y.append(potential_y_values[train_Y[index]])
    for index in range(len(test_Y)):
        new_test_Y.append(potential_y_values[test_Y[index]])
        

In [60]:
# build the logistic regression model - need clean data...
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(multi_class='ovr')
LR_model.fit(train_X, train_Y)

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/sta

LogisticRegression(multi_class='ovr')

In [61]:
# confusion matrix
from sklearn.metrics import confusion_matrix

pred_Y = LR_model.predict(test_X);
confusion_matrix(test_Y, pred_Y)

array([[67,  0,  0],
       [ 0, 56,  0],
       [ 0,  0, 87]])

In [62]:
# evaluate the accuracy of the LR model
accuracy = LR_model.score(test_X, test_Y)
print(accuracy)

1.0


In [63]:
# build the ANN model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

ANN_model = keras.Sequential()

# add hidden layers
for i in range(num_hidden_layers):
    if i == 0:
        ANN_model.add(Dense(num_hidden_layer_nodes[i], input_dim = train_X.shape[1], activation=hidden_layer_activations[i]))
    ANN_model.add(Dense(num_hidden_layer_nodes[i], activation=hidden_layer_activations[i]))

# add output layers
ANN_model.add(Dense(1 if original_trend_values else num_clusters, activation=hidden_layer_activations[num_hidden_layers]))

ANN_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
ANN_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 20)                720       
_________________________________________________________________
dense_13 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_14 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_15 (Dense)             (None, 30)                630       
_________________________________________________________________
dense_16 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 63        
Total params: 2,873
Trainable params: 2,873
Non-trainable params: 0
____________________________________________________

In [64]:
# train the model
ANN_model.fit(train_X, np.array(new_train_Y), validation_data=(test_X, np.array(new_test_Y)), epochs=epochs, batch_size=batch_size)

Train on 488 samples, validate on 210 samples
Epoch 1/300
488/488 [==============================] - 1s 2ms/sample - loss: 0.2247 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.2244 - val_accuracy: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 2/300
488/488 [==============================] - 0s 87us/sample - loss: 0.2231 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.2223 - val_accuracy: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 3/300
488/488 [==============================] - 0s 83us/sample - loss: 0.2222 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.2209 - val_accuracy: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 4/300
488/488 [==============================] - 0s 79us/sample - loss: 0.2216 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.2202

Epoch 32/300
488/488 [==============================] - 0s 73us/sample - loss: 0.2207 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.2193 - val_accuracy: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 33/300
488/488 [==============================] - 0s 65us/sample - loss: 0.2207 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.2192 - val_accuracy: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 34/300
488/488 [==============================] - 0s 48us/sample - loss: 0.2206 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.2190 - val_accuracy: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 35/300
488/488 [==============================] - 0s 56us/sample - loss: 0.2205 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.2187 - val_accuracy: 0.0000e+00 - val_recall_

488/488 [==============================] - 0s 68us/sample - loss: 0.1996 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.1974 - val_accuracy: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 64/300
488/488 [==============================] - 0s 75us/sample - loss: 0.1970 - accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - val_loss: 0.1949 - val_accuracy: 0.0000e+00 - val_recall_5: 0.0095 - val_precision_5: 1.0000
Epoch 65/300
488/488 [==============================] - 0s 60us/sample - loss: 0.1942 - accuracy: 0.0000e+00 - recall_5: 0.0471 - precision_5: 1.0000 - val_loss: 0.1921 - val_accuracy: 0.0000e+00 - val_recall_5: 0.1571 - val_precision_5: 1.0000
Epoch 66/300
488/488 [==============================] - 0s 68us/sample - loss: 0.1911 - accuracy: 0.0000e+00 - recall_5: 0.1352 - precision_5: 1.0000 - val_loss: 0.1890 - val_accuracy: 0.0000e+00 - val_recall_5: 0.1905 - val_precision_5: 0.9756
Epoch 67/

488/488 [==============================] - 0s 59us/sample - loss: 0.1229 - accuracy: 0.0000e+00 - recall_5: 0.5861 - precision_5: 0.6515 - val_loss: 0.1261 - val_accuracy: 0.0000e+00 - val_recall_5: 0.6048 - val_precision_5: 0.6865
Epoch 97/300
488/488 [==============================] - 0s 64us/sample - loss: 0.1223 - accuracy: 0.0000e+00 - recall_5: 0.5861 - precision_5: 0.6515 - val_loss: 0.1254 - val_accuracy: 0.0000e+00 - val_recall_5: 0.6095 - val_precision_5: 0.6919
Epoch 98/300
488/488 [==============================] - 0s 48us/sample - loss: 0.1218 - accuracy: 0.0000e+00 - recall_5: 0.5861 - precision_5: 0.6500 - val_loss: 0.1247 - val_accuracy: 0.0000e+00 - val_recall_5: 0.6095 - val_precision_5: 0.6919
Epoch 99/300
488/488 [==============================] - 0s 62us/sample - loss: 0.1212 - accuracy: 0.0000e+00 - recall_5: 0.5861 - precision_5: 0.6500 - val_loss: 0.1241 - val_accuracy: 0.0000e+00 - val_recall_5: 0.6143 - val_precision_5: 0.6935
Epoch 100/300
488/488 [==========

488/488 [==============================] - 0s 75us/sample - loss: 0.1110 - accuracy: 0.0000e+00 - recall_5: 0.9119 - precision_5: 0.7417 - val_loss: 0.1139 - val_accuracy: 0.0000e+00 - val_recall_5: 0.8952 - val_precision_5: 0.7611
Epoch 130/300
488/488 [==============================] - 0s 57us/sample - loss: 0.1107 - accuracy: 0.0000e+00 - recall_5: 0.9098 - precision_5: 0.7388 - val_loss: 0.1134 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9143 - val_precision_5: 0.7589
Epoch 131/300
488/488 [==============================] - 0s 50us/sample - loss: 0.1103 - accuracy: 0.0000e+00 - recall_5: 0.9488 - precision_5: 0.7408 - val_loss: 0.1132 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9429 - val_precision_5: 0.7472
Epoch 132/300
488/488 [==============================] - 0s 54us/sample - loss: 0.1098 - accuracy: 0.0000e+00 - recall_5: 0.9713 - precision_5: 0.7337 - val_loss: 0.1129 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9476 - val_precision_5: 0.7316
Epoch 133/300
488/488 [=======

488/488 [==============================] - 0s 60us/sample - loss: 0.0777 - accuracy: 0.0000e+00 - recall_5: 0.9508 - precision_5: 0.9355 - val_loss: 0.0815 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9238 - val_precision_5: 0.9108
Epoch 163/300
488/488 [==============================] - 0s 58us/sample - loss: 0.0761 - accuracy: 0.0000e+00 - recall_5: 0.9508 - precision_5: 0.9393 - val_loss: 0.0797 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9238 - val_precision_5: 0.9151
Epoch 164/300
488/488 [==============================] - 0s 50us/sample - loss: 0.0741 - accuracy: 0.0000e+00 - recall_5: 0.9549 - precision_5: 0.9414 - val_loss: 0.0787 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9190 - val_precision_5: 0.9234
Epoch 165/300
488/488 [==============================] - 0s 53us/sample - loss: 0.0724 - accuracy: 0.0000e+00 - recall_5: 0.9529 - precision_5: 0.9413 - val_loss: 0.0767 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9190 - val_precision_5: 0.9234
Epoch 166/300
488/488 [=======

488/488 [==============================] - 0s 64us/sample - loss: 0.0333 - accuracy: 0.0000e+00 - recall_5: 0.9877 - precision_5: 0.9877 - val_loss: 0.0439 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9476 - val_precision_5: 0.9476
Epoch 196/300
488/488 [==============================] - 0s 85us/sample - loss: 0.0326 - accuracy: 0.0000e+00 - recall_5: 0.9775 - precision_5: 0.9775 - val_loss: 0.0427 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9524 - val_precision_5: 0.9524
Epoch 197/300
488/488 [==============================] - 0s 76us/sample - loss: 0.0317 - accuracy: 0.0000e+00 - recall_5: 0.9857 - precision_5: 0.9816 - val_loss: 0.0418 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9429 - val_precision_5: 0.9519
Epoch 198/300
488/488 [==============================] - 0s 68us/sample - loss: 0.0318 - accuracy: 0.0000e+00 - recall_5: 0.9795 - precision_5: 0.9835 - val_loss: 0.0410 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9571 - val_precision_5: 0.9571
Epoch 199/300
488/488 [=======

488/488 [==============================] - 0s 90us/sample - loss: 0.0177 - accuracy: 0.0000e+00 - recall_5: 0.9959 - precision_5: 0.9959 - val_loss: 0.0324 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9571 - val_precision_5: 0.9571
Epoch 229/300
488/488 [==============================] - 0s 84us/sample - loss: 0.0174 - accuracy: 0.0000e+00 - recall_5: 0.9959 - precision_5: 0.9959 - val_loss: 0.0321 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9524 - val_precision_5: 0.9569
Epoch 230/300
488/488 [==============================] - 0s 71us/sample - loss: 0.0172 - accuracy: 0.0000e+00 - recall_5: 0.9939 - precision_5: 0.9939 - val_loss: 0.0319 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9524 - val_precision_5: 0.9524
Epoch 231/300
488/488 [==============================] - 0s 89us/sample - loss: 0.0170 - accuracy: 0.0000e+00 - recall_5: 0.9939 - precision_5: 0.9939 - val_loss: 0.0315 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9524 - val_precision_5: 0.9524
Epoch 232/300
488/488 [=======

488/488 [==============================] - 0s 86us/sample - loss: 0.0112 - accuracy: 0.0000e+00 - recall_5: 1.0000 - precision_5: 1.0000 - val_loss: 0.0321 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9476 - val_precision_5: 0.9476
Epoch 262/300
488/488 [==============================] - 0s 85us/sample - loss: 0.0111 - accuracy: 0.0000e+00 - recall_5: 1.0000 - precision_5: 1.0000 - val_loss: 0.0319 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9429 - val_precision_5: 0.9429
Epoch 263/300
488/488 [==============================] - 0s 127us/sample - loss: 0.0109 - accuracy: 0.0000e+00 - recall_5: 1.0000 - precision_5: 1.0000 - val_loss: 0.0319 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9429 - val_precision_5: 0.9429
Epoch 264/300
488/488 [==============================] - 0s 61us/sample - loss: 0.0106 - accuracy: 0.0000e+00 - recall_5: 1.0000 - precision_5: 1.0000 - val_loss: 0.0321 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9476 - val_precision_5: 0.9476
Epoch 265/300
488/488 [======

488/488 [==============================] - 0s 81us/sample - loss: 0.0077 - accuracy: 0.0000e+00 - recall_5: 1.0000 - precision_5: 1.0000 - val_loss: 0.0314 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9429 - val_precision_5: 0.9429
Epoch 295/300
488/488 [==============================] - 0s 101us/sample - loss: 0.0076 - accuracy: 0.0000e+00 - recall_5: 1.0000 - precision_5: 1.0000 - val_loss: 0.0314 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9429 - val_precision_5: 0.9429
Epoch 296/300
488/488 [==============================] - 0s 82us/sample - loss: 0.0075 - accuracy: 0.0000e+00 - recall_5: 1.0000 - precision_5: 1.0000 - val_loss: 0.0319 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9429 - val_precision_5: 0.9429
Epoch 297/300
488/488 [==============================] - 0s 81us/sample - loss: 0.0075 - accuracy: 0.0000e+00 - recall_5: 1.0000 - precision_5: 1.0000 - val_loss: 0.0316 - val_accuracy: 0.0000e+00 - val_recall_5: 0.9429 - val_precision_5: 0.9429
Epoch 298/300
488/488 [======

In [65]:
# confusion matrix
pred_Y = ANN_model.predict_classes(test_X);
confusion_matrix(test_Y, pred_Y)

array([[64,  0,  3],
       [ 0, 53,  3],
       [ 1,  5, 81]])

In [66]:
# report evaluation metrics 
evaluated_metrics = ANN_model.evaluate(test_X, np.array(new_test_Y))
for i in range(len(metrics)):
    print(metrics_names[i] + ": %.2f" % evaluated_metrics[i])

210/210 [==============================] - 0s 106us/sample - loss: 0.0317 - accuracy: 0.0000e+00 - recall_5: 0.9429 - precision_5: 0.9429
accuracy: 0.03
recall: 0.00
precision: 0.94


In [67]:
df

,title,channelTitle,duration,tags,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,Channel_videoCount,Channel_description,view_count_update_11_19_14,likes_update_11_19_14,dislikes_update_11_19_14,comment_count_update_11_19_14,engagement_rate,INTL,UNK,USA,Category_24.0,Category_25.0,Category_26.0,Category_17.0,Category_22.0,Category_20.0,Category_1.0,Category_10.0,Category_23.0,Category_28.0,Category_2.0,Category_27.0,Category_19.0,Category_15.0,Category_29.0,Category_30.0,cluster
0,0.000,0.0,-0.0707230866960673,36.0,0,0.034,0.819940,0.504608,0,-0.190170,0.122,6.152175,3.391273,10.316363,2.855296,0.028492,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000,0.0,-0.0856798827419832,28.0,0,0.048,-0.274970,-0.350255,0,1.155772,0.000,3.001911,0.578677,4.999903,3.142250,0.090361,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000,0.0,0.09488158288091483,13.0,0,0.062,0.650191,0.461214,0,1.266836,0.107,1.368339,0.537524,2.314506,1.066382,0.006721,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,0.274,0.0,-0.07104706061763949,0.0,0,0.276,-0.258217,0.002973,0,-0.408313,0.000,1.882086,2.655548,2.774959,0.721674,0.045112,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.000,0.0,-0.07288291283988188,12.0,0,0.000,-0.330711,-0.407708,0,-0.204876,0.188,0.280922,-0.071268,0.049159,0.022729,0.029918,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,0.000,0.0,0.00740862405642476,15.0,1,0.000,-0.334392,-0.425760,0,-0.326094,0.231,-0.214186,-0.199963,-0.207538,-0.129468,0.000022,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
785,0.000,0.0,-0.07822848254582293,5.0,0,0.000,-0.295265,-0.263467,0,-0.244893,0.000,0.043742,-0.111971,-0.070656,-0.069428,0.003647,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
786,0.000,0.0,-0.091727395944664,0.0,0,0.000,-0.317183,-0.415519,0,-0.193704,0.238,-0.194914,-0.197159,-0.199876,-0.124227,0.000930,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
787,0.000,0.0,-0.09226735248061764,6.0,0,0.172,-0.050133,-0.048232,0,0.407770,0.197,-0.194899,-0.182087,0.034182,-0.021491,0.000558,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
